# 🎤 DOWNLOAD E PREPARAZIONE DATASET ITALIANO PER PIPER
## Dataset: giacomoarienti/female-LJSpeech-italian

**Questo notebook scarica il dataset italiano e lo salva in Google Drive**

---

### Cosa fa questo notebook:
- ✅ Scarica 5856 file audio in italiano da Hugging Face
- ✅ Converte tutto a 16000Hz, Mono, 16-bit
- ✅ Salva in Google Drive (riutilizzabile per sempre)
- ✅ Verifica completezza e correttezza del dataset

### Risultato finale:
- 📁 Directory: `/content/drive/MyDrive/ljspeech_italian/`
- 🎵 5856 file audio (8h 23m totali)
- 📄 metadata.csv con tutte le trascrizioni
- 💾 Salvato permanentemente in Google Drive (~1.5 GB)

---

## 📂 STEP 0: Monta Google Drive

**IMPORTANTE:** Clicca sul link che appare e autorizza l'accesso a Google Drive

In [ ]:
from google.colab import drive

print("📂 Montaggio Google Drive...\n")
drive.mount('/content/drive')
print("\n✅ Google Drive montato con successo!")

📂 Montaggio Google Drive...

Mounted at /content/drive

✅ Google Drive montato con successo!


## 📦 STEP 1: Installazione dipendenze

In [ ]:
print("📦 Installazione dipendenze...")

!pip install -q datasets librosa soundfile pandas tqdm

print("✅ Dipendenze installate!\n")

📦 Installazione dipendenze...
✅ Dipendenze installate!



## 📥 STEP 2: Download dataset italiano

In [ ]:
from datasets import load_dataset, Audio
import os

print("="*60)
print("  📥 DOWNLOAD DATASET ITALIANO")
print("="*60)

print("\n📚 Scarico dataset 'giacomoarienti/female-LJSpeech-italian'...")
print("⏱️  Questo richiederà alcuni minuti...\n")

try:
    # Carica dataset SENZA decodificare automaticamente l'audio
    dataset = load_dataset("giacomoarienti/female-LJSpeech-italian", split="train")

    # Rimuovi il decoding automatico dell'audio
    dataset = dataset.cast_column("audio", Audio(decode=False))

    print(f"✅ Dataset scaricato: {len(dataset)} campioni audio\n")

except Exception as e:
    print(f"❌ Errore durante il download: {e}")
    raise

  📥 DOWNLOAD DATASET ITALIANO

📚 Scarico dataset 'giacomoarienti/female-LJSpeech-italian'...
⏱️  Questo richiederà alcuni minuti...



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/426 [00:00<?, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/398M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/397M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5856 [00:00<?, ? examples/s]

✅ Dataset scaricato: 5856 campioni audio



## 📁 STEP 3: Creazione struttura directory in Google Drive

In [ ]:
print("="*60)
print("  📁 CREAZIONE STRUTTURA DATASET")
print("="*60)

# IMPORTANTE: Salva in Google Drive!
dataset_dir = "/content/drive/MyDrive/ljspeech_italian"
wavs_dir = f"{dataset_dir}/wavs"

# Crea directory
os.makedirs(wavs_dir, exist_ok=True)
print(f"\n✅ Directory create in Google Drive:")
print(f"   📁 {dataset_dir}/")
print(f"   📁 {wavs_dir}/")
print(f"\n💾 Il dataset sarà salvato permanentemente in Google Drive!\n")

  📁 CREAZIONE STRUTTURA DATASET

✅ Directory create in Google Drive:
   📁 /content/drive/MyDrive/ljspeech_italian/
   📁 /content/drive/MyDrive/ljspeech_italian/wavs/

💾 Il dataset sarà salvato permanentemente in Google Drive!



## 🔊 STEP 4: Conversione e salvataggio audio

**Nota:** Questo step richiede 5-10 minuti. I file vengono salvati direttamente in Google Drive.

In [ ]:
import soundfile as sf
import librosa
import numpy as np
from tqdm import tqdm
import re
import io

print("="*60)
print("  🔊 CONVERSIONE AUDIO E TRASCRIZIONI")
print("="*60)

# Debug: mostra struttura del primo elemento
print("\n🔍 Debug: struttura dataset...")
first_item = dataset[0]
print(f"Campi disponibili: {list(first_item.keys())}")
if 'audio' in first_item:
    print(f"Campi audio: {list(first_item['audio'].keys())}")
print()

metadata_lines = []
errors = 0
success = 0

print("⚙️  Conversione in corso (16000Hz, Mono, 16-bit)...")
print("💾 Salvataggio in Google Drive...\n")

for idx, item in enumerate(tqdm(dataset, desc="Processando")):
    try:
        # Nome file
        if 'id' in item and item['id']:
            filename = item['id']
        else:
            filename = f"audio_{idx:06d}"

        wav_path = f"{wavs_dir}/{filename}.wav"

        # Ottieni i bytes dell'audio
        audio_bytes = item['audio']['bytes']

        # Carica audio dai bytes
        audio, sr_original = sf.read(io.BytesIO(audio_bytes))

        # Converti a mono se stereo
        if len(audio.shape) > 1:
            audio = audio.mean(axis=1)

        # Resample a 16000Hz se necessario
        if sr_original != 16000:
            audio = librosa.resample(audio, orig_sr=sr_original, target_sr=16000)

        # Normalizza audio
        if np.max(np.abs(audio)) > 0:
            audio = audio / np.max(np.abs(audio)) * 0.95

        # Salva come WAV (16000Hz, Mono, 16-bit)
        sf.write(wav_path, audio, 16000, subtype='PCM_16')

        # Ottieni trascrizione - PROVA TUTTI I CAMPI POSSIBILI
        text = None
        for field in ['text', 'sentence', 'transcription', 'transcript', 'normalized_text']:
            if field in item and item[field]:
                text = str(item[field]).strip()
                if text:  # Verifica che non sia stringa vuota
                    break

        # Debug: mostra primi 3 errori di testo mancante
        if not text:
            errors += 1
            if errors <= 3:
                print(f"\n⚠️  File {idx} ({filename}): testo non trovato!")
                print(f"   Campi disponibili: {list(item.keys())}")
            if os.path.exists(wav_path):
                os.remove(wav_path)
            continue

        # Pulizia testo
        text = text.replace('\n', ' ').replace('\r', ' ')
        text = re.sub(r'\s+', ' ', text)

        # Aggiungi a metadata
        if len(text) > 0:
            metadata_lines.append(f"{filename}|{text}")
            success += 1

            # Debug: mostra prime 3 trascrizioni
            if success <= 3:
                print(f"\n✅ {filename}: {text[:60]}...")
        else:
            errors += 1
            if os.path.exists(wav_path):
                os.remove(wav_path)

    except Exception as e:
        errors += 1
        if errors <= 5:
            print(f"\n❌ Errore file {idx}: {e}")
        if 'wav_path' in locals() and os.path.exists(wav_path):
            try:
                os.remove(wav_path)
            except:
                pass

print(f"\n✅ Conversione completata!")
print(f"   ✓ File salvati: {success}")
print(f"   ✗ Errori: {errors}")
print(f"   💾 Tutto salvato in Google Drive!")

# IMPORTANTE: Verifica che abbiamo trascrizioni!
if len(metadata_lines) == 0:
    print("\n❌ ERRORE CRITICO: Nessuna trascrizione estratta!")
    print("   Il dataset potrebbe non avere il campo testo corretto.")
    raise ValueError("Nessuna trascrizione estratta dal dataset!")
else:
    print(f"\n✅ {len(metadata_lines)} trascrizioni pronte per il salvataggio!")

  🔊 CONVERSIONE AUDIO E TRASCRIZIONI

🔍 Debug: struttura dataset...
Campi disponibili: ['text', 'audio']
Campi audio: ['bytes', 'path']

⚙️  Conversione in corso (16000Hz, Mono, 16-bit)...
💾 Salvataggio in Google Drive...



Processando:   0%|          | 6/5856 [00:00<01:41, 57.68it/s]


✅ audio_000000: e Antoni non aveva voglia di cantare, col cappuccio sul naso...

✅ audio_000001: - Questo qui ha più giudizio del grande! - diceva la cugina ...

✅ audio_000002: - Ma ancora cè tempo. - Sì, - affermò Alessi;...


Processando: 100%|██████████| 5856/5856 [02:08<00:00, 45.53it/s]


✅ Conversione completata!
   ✓ File salvati: 5856
   ✗ Errori: 0
   💾 Tutto salvato in Google Drive!

✅ 5856 trascrizioni pronte per il salvataggio!


## 💾 STEP 5: Salva metadata.csv

In [ ]:
print("="*60)
print("  💾 SALVATAGGIO METADATA")
print("="*60)

metadata_path = f"{dataset_dir}/metadata.csv"

with open(metadata_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(metadata_lines))

print(f"\n✅ metadata.csv salvato: {len(metadata_lines)} righe")
print(f"   📁 {metadata_path}\n")

  💾 SALVATAGGIO METADATA

✅ metadata.csv salvato: 5856 righe
   📁 /content/drive/MyDrive/ljspeech_italian/metadata.csv



## 🔍 STEP 6: VERIFICA COMPLETA DATASET

In [ ]:
import pandas as pd
import wave
from pathlib import Path
import random

print("="*60)
print("  🔍 VERIFICA DATASET")
print("="*60)

# Verifica metadata
metadata = pd.read_csv(metadata_path, sep='|', header=None, names=['filename', 'text'])
num_metadata = len(metadata)

# Verifica WAV
wav_files = list(Path(wavs_dir).glob("*.wav"))
num_wavs = len(wav_files)

print(f"\n📊 Statistiche:")
print(f"   📄 Righe metadata: {num_metadata}")
print(f"   🎵 File WAV: {num_wavs}")

if num_metadata == num_wavs:
    print(f"   ✅ Corrispondenza perfetta!")
else:
    print(f"   ⚠️  Discrepanza: {abs(num_metadata - num_wavs)} file")

# Verifica formato audio su 5 file casuali
print(f"\n🔊 Test 5 file audio casuali:")

sample_files = random.sample(wav_files, min(5, len(wav_files)))
all_correct = True

for wav_path in sample_files:
    try:
        with wave.open(str(wav_path), 'rb') as wav:
            sr = wav.getframerate()
            channels = wav.getnchannels()
            sampwidth = wav.getsampwidth()
            frames = wav.getnframes()
            duration = frames / sr

            is_correct = (sr == 16000 and channels == 1 and sampwidth == 2)
            status = "✅" if is_correct else "❌"

            print(f"{status} {wav_path.name}: {sr}Hz, {channels}ch, {sampwidth*8}bit, {duration:.2f}s")

            if not is_correct:
                all_correct = False
    except Exception as e:
        print(f"❌ {wav_path.name}: Errore - {e}")
        all_correct = False

# Mostra prime 5 trascrizioni
print(f"\n📝 Prime 5 trascrizioni:")
for idx, row in metadata.head(5).iterrows():
    text_preview = row['text'][:70] + "..." if len(row['text']) > 70 else row['text']
    print(f"   {row['filename']}: {text_preview}")

# Calcola durata totale
print(f"\n⏱️  Calcolo durata totale...")
total_duration = 0
sample_size = min(100, len(wav_files))

for wav_file in tqdm(wav_files[:sample_size], desc="Campionamento"):
    try:
        with wave.open(str(wav_file), 'rb') as wav:
            frames = wav.getnframes()
            rate = wav.getframerate()
            total_duration += frames / rate
    except:
        pass

# Estrapola durata totale
if sample_size > 0:
    avg_duration = total_duration / sample_size
    estimated_total = avg_duration * len(wav_files)

    hours = int(estimated_total // 3600)
    minutes = int((estimated_total % 3600) // 60)
    seconds = int(estimated_total % 60)

    print(f"\n📊 Durata stimata totale: {hours}h {minutes}m {seconds}s")
    print(f"📊 Durata media per file: {avg_duration:.2f}s")

# RIEPILOGO FINALE
print("\n" + "="*60)
print("  📊 RIEPILOGO FINALE")
print("="*60)

if all_correct and num_metadata == num_wavs and num_wavs > 0:
    print("\n🎉 DATASET PRONTO PER IL TRAINING!")
    print(f"\n✅ Tutto corretto:")
    print(f"   ✓ Dataset: giacomoarienti/female-LJSpeech-italian")
    print(f"   ✓ {num_wavs} file audio")
    print(f"   ✓ {num_metadata} trascrizioni")
    print(f"   ✓ Formato: 16000Hz, Mono, 16-bit")
    if 'hours' in locals():
        print(f"   ✓ Durata: ~{hours}h {minutes}m")
    print(f"\n💾 SALVATO IN GOOGLE DRIVE:")
    print(f"   📁 {dataset_dir}")
    print(f"\n💡 IMPORTANTE:")
    print(f"   • Il dataset è ora permanentemente salvato in Google Drive")
    print(f"   • Puoi chiudere Colab quando vuoi")
    print(f"   • Per il training, usa il path: {dataset_dir}")
else:
    print("\n⚠️  Alcuni problemi rilevati:")
    if not all_correct:
        print("   • Formato audio non uniforme")
    if num_metadata != num_wavs:
        print(f"   • Discrepanza metadata/audio: {num_metadata} vs {num_wavs}")
    print("\nVerifica i dettagli sopra prima di procedere.")

print("\n" + "="*60)

  🔍 VERIFICA DATASET

📊 Statistiche:
   📄 Righe metadata: 5856
   🎵 File WAV: 5856
   ✅ Corrispondenza perfetta!

🔊 Test 5 file audio casuali:
✅ audio_004884.wav: 16000Hz, 1ch, 16bit, 5.59s
✅ audio_001039.wav: 16000Hz, 1ch, 16bit, 5.00s
✅ audio_003752.wav: 16000Hz, 1ch, 16bit, 5.55s
✅ audio_005223.wav: 16000Hz, 1ch, 16bit, 3.14s
✅ audio_001077.wav: 16000Hz, 1ch, 16bit, 4.87s

📝 Prime 5 trascrizioni:
   audio_000000: e Antoni non aveva voglia di cantare, col cappuccio sul naso, e gli to...
   audio_000001: - Questo qui ha più giudizio del grande! - diceva la cugina Anna.
   audio_000002: - Ma ancora cè tempo. - Sì, - affermò Alessi;
   audio_000003: so che non ve le devo io.
   audio_000004: o lo zio Crocifisso, che ci aveva la sua roba di qua e di là

⏱️  Calcolo durata totale...


Campionamento: 100%|██████████| 100/100 [01:13<00:00,  1.35it/s]


📊 Durata stimata totale: 8h 6m 26s
📊 Durata media per file: 4.98s

  📊 RIEPILOGO FINALE

🎉 DATASET PRONTO PER IL TRAINING!

✅ Tutto corretto:
   ✓ Dataset: giacomoarienti/female-LJSpeech-italian
   ✓ 5856 file audio
   ✓ 5856 trascrizioni
   ✓ Formato: 16000Hz, Mono, 16-bit
   ✓ Durata: ~8h 6m

💾 SALVATO IN GOOGLE DRIVE:
   📁 /content/drive/MyDrive/ljspeech_italian

💡 IMPORTANTE:
   • Il dataset è ora permanentemente salvato in Google Drive
   • Puoi chiudere Colab quando vuoi
   • Per il training, usa il path: /content/drive/MyDrive/ljspeech_italian



In [ ]:

import os
import pandas as pd
from pathlib import Path

print("="*60)
print("  🔍 VERIFICA METADATI E DATASET")
print("="*60)

# Path dataset in Google Drive
dataset_dir = "/content/drive/MyDrive/ljspeech_italian"
metadata_path = f"{dataset_dir}/metadata.csv"
wavs_dir = f"{dataset_dir}/wavs"

# 1. Verifica esistenza metadata.csv
print("\n1️⃣ Verifica file metadata.csv...")
if os.path.exists(metadata_path):
    file_size = os.path.getsize(metadata_path) / (1024 * 1024)  # MB
    print(f"   ✅ File trovato: {metadata_path}")
    print(f"   💾 Dimensione: {file_size:.2f} MB")
else:
    print(f"   ❌ File NON trovato: {metadata_path}")
    print("   Devi rieseguire il notebook di download!")
    exit()

# 2. Carica e analizza metadata.csv
print("\n2️⃣ Analisi contenuto metadata.csv...")
try:
    # Leggi il file
    metadata = pd.read_csv(metadata_path, sep='|', header=None, names=['filename', 'text'])
    num_lines = len(metadata)
    print(f"   ✅ Righe totali: {num_lines}")

    # Verifica formato
    if metadata.shape[1] == 2:
        print(f"   ✅ Formato corretto: 2 colonne (filename|text)")
    else:
        print(f"   ❌ Formato errato: {metadata.shape[1]} colonne invece di 2!")

except Exception as e:
    print(f"   ❌ Errore lettura file: {e}")
    exit()

# 3. Mostra prime 5 trascrizioni
print("\n3️⃣ Prime 5 trascrizioni:")
for idx, row in metadata.head(5).iterrows():
    text_preview = row['text'][:60] + "..." if len(row['text']) > 60 else row['text']
    print(f"   {idx+1}. {row['filename']}: {text_preview}")

# 4. Statistiche testo
print("\n4️⃣ Statistiche trascrizioni:")
text_lengths = metadata['text'].str.len()
print(f"   📊 Lunghezza media: {text_lengths.mean():.0f} caratteri")
print(f"   📊 Lunghezza min: {text_lengths.min()} caratteri")
print(f"   📊 Lunghezza max: {text_lengths.max()} caratteri")

# Verifica righe problematiche
empty_text = metadata[metadata['text'].str.strip() == '']
if len(empty_text) > 0:
    print(f"   ⚠️  {len(empty_text)} righe con testo vuoto!")
else:
    print(f"   ✅ Nessuna riga con testo vuoto")

# 5. Verifica corrispondenza con file audio
print("\n5️⃣ Verifica corrispondenza con file WAV...")
if os.path.exists(wavs_dir):
    wav_files = list(Path(wavs_dir).glob("*.wav"))
    num_wavs = len(wav_files)
    print(f"   📄 File in metadata.csv: {num_lines}")
    print(f"   🎵 File WAV in wavs/: {num_wavs}")

    if num_lines == num_wavs:
        print(f"   ✅ Corrispondenza PERFETTA!")
    else:
        diff = abs(num_lines - num_wavs)
        print(f"   ⚠️  Discrepanza: {diff} file di differenza!")

    # Verifica che i filename nel metadata esistano come WAV
    metadata_files = set(metadata['filename'].astype(str))
    actual_files = set(f.stem for f in wav_files)

    missing = metadata_files - actual_files
    if len(missing) > 0:
        print(f"   ⚠️  {len(missing)} file citati in metadata ma mancanti in wavs/")
        print(f"      Esempi: {list(missing)[:3]}")
    else:
        print(f"   ✅ Tutti i file citati in metadata esistono!")

else:
    print(f"   ❌ Cartella wavs/ non trovata!")

# 6. Riepilogo finale
print("\n" + "="*60)
print("  📊 RIEPILOGO FINALE")
print("="*60)

if os.path.exists(metadata_path) and num_lines > 0 and num_lines == num_wavs:
    print("\n🎉 TUTTO PERFETTO!")
    print(f"   ✅ metadata.csv: {num_lines} righe")
    print(f"   ✅ File WAV: {num_wavs}")
    print(f"   ✅ Corrispondenza perfetta")
    print(f"\n💾 Dataset completo salvato in Google Drive!")
    print(f"   📁 {dataset_dir}")
    print(f"\n🚀 Sei pronto per il training!")
else:
    print("\n⚠️  Ci sono alcuni problemi:")
    if num_lines == 0:
        print("   • metadata.csv è vuoto!")
    if num_lines != num_wavs:
        print(f"   • Discrepanza file: {num_lines} vs {num_wavs}")
    print("\n💡 Potrebbe essere necessario rieseguire il download.")

print("\n" + "="*60)

  🔍 VERIFICA METADATI E DATASET

1️⃣ Verifica file metadata.csv...
   ✅ File trovato: /content/drive/MyDrive/ljspeech_italian/metadata.csv
   💾 Dimensione: 0.53 MB

2️⃣ Analisi contenuto metadata.csv...
   ✅ Righe totali: 5856
   ✅ Formato corretto: 2 colonne (filename|text)

3️⃣ Prime 5 trascrizioni:
   1. audio_000000: e Antoni non aveva voglia di cantare, col cappuccio sul naso...
   2. audio_000001: - Questo qui ha più giudizio del grande! - diceva la cugina ...
   3. audio_000002: - Ma ancora cè tempo. - Sì, - affermò Alessi;
   4. audio_000003: so che non ve le devo io.
   5. audio_000004: o lo zio Crocifisso, che ci aveva la sua roba di qua e di là

4️⃣ Statistiche trascrizioni:
   📊 Lunghezza media: 81 caratteri
   📊 Lunghezza min: 11 caratteri
   📊 Lunghezza max: 166 caratteri
   ✅ Nessuna riga con testo vuoto

5️⃣ Verifica corrispondenza con file WAV...
   📄 File in metadata.csv: 5856
   🎵 File WAV in wavs/: 5856
   ✅ Corrispondenza PERFETTA!
   ✅ Tutti i file citati in metad